This workbench require installation of the following packages: tensorflow, torch and py3nvml
It is based on: https://github.com/redhat-gpte-devopsautomation/rhods-gpu-testing/blob/main/gpu-testing.ipynb

In [ ]:
!pip install tensorflow py3nvml torch
#!pip --index-url <internal-artifact-registry-url> pip install tensorflow py3nvml torch

Run a sample program using tensorflow library to test GPU's available and how many for this notebook?

In [ ]:
import tensorflow as tf

# Check if a GPU is available
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# Get the number of available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Number of available GPUs: {len(gpus)}")
else:
    print("No GPUs found")


Use py3nvml to get details about the GPUs.

In [ ]:
import py3nvml

# Initialize the NVML library
py3nvml.py3nvml.nvmlInit()

# Get the number of available GPUs
num_gpus = py3nvml.py3nvml.nvmlDeviceGetCount()

# Loop through all available GPUs and print their information
for i in range(num_gpus):
    handle = py3nvml.py3nvml.nvmlDeviceGetHandleByIndex(i)
    print(f"GPU {i}: {py3nvml.py3nvml.nvmlDeviceGetName(handle)}")
    print(f"Memory usage: {py3nvml.py3nvml.nvmlDeviceGetMemoryInfo(handle).used / 1024**2:.2f} MB")
    print(f"Temperature: {py3nvml.py3nvml.nvmlDeviceGetTemperature(handle, py3nvml.py3nvml.NVML_TEMPERATURE_GPU)} C")
    print(f"Power usage: {py3nvml.py3nvml.nvmlDeviceGetPowerUsage(handle) / 1000:.2f} W")
    print()


Use subprocess to get mmore details about NVIDIA GPU's using nvidia-smi cli.

In [ ]:
output = subprocess.check_output(['nvidia-smi'])

# Print the output
print(output.decode('utf-8'))


Run a sample neural network based program to rn on GPU's when detected.

In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create a random input tensor
input_tensor = torch.randn(1, 10)

# Move the input tensor to the GPU
input_tensor = input_tensor.to(device)

# Create a neural network and move it to the GPU
net = Net().to(device)

# Define a loss function and an optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Train the neural network
for i in range(100):
    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    output_tensor = net(input_tensor)

    # Compute the loss
    target_tensor = torch.randn(1, 1)
    target_tensor = target_tensor.to(device)
    loss = criterion(output_tensor, target_tensor)

    # Backward pass
    loss.backward()

    # Update the weights
    optimizer.step()

# Print the trained model
print(net)

